In [14]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import sys
sys.path.append("..")
from src.data import metadata

In [32]:
metadata_df = metadata.build_df()

In [33]:
metadata_df

,KNUMBER,APPLICANT,CONTACT,STREET1,STREET2,CITY,STATE,COUNTRY_CODE,ZIP,POSTAL_CODE,...,DECISION,REVIEWADVISECOMM,PRODUCTCODE,STATEORSUMM,CLASSADVISECOMM,SSPINDICATOR,TYPE,THIRDPARTY,EXPEDITEDREVIEW,DEVICENAME
0,K760001,"ZIMMER, INC.",NaN,"4221 Richmond Rd., N.W.",NaN,Walker,MI,US,49534,49534,...,SESE,PM,NaN,NaN,NaN,NaN,Traditional,N,NaN,ARCH SUPPORT (ARCH AID)
1,K760002,"ZIMMER, INC.",NaN,NaN,NaN,NaN,MO,US,NaN,NaN,...,SESE,PM,IQI,NaN,PM,NaN,Traditional,N,NaN,KNEE AID
2,K760003,"ZIMMER, INC.",NaN,803 N. Front St. Suite 3,NaN,McHenry,IL,US,60050,60050,...,SESE,PM,ITG,NaN,PM,NaN,Traditional,N,NaN,CAST MATERIAL (WICKET STOCKINETTE)
3,K760004,STEWART-NAUMANN LABORATORIES,NaN,803 N. Front St. Suite 3,NaN,McHenry,IL,US,60050,60050,...,SESE,HO,FMF,NaN,HO,NaN,Traditional,N,NaN,"SYRINGE, DISPOSABLE, ALL PLASTIC"
4,K760005,STEWART-NAUMANN LABORATORIES,NaN,803 N. Front St. Suite 3,NaN,McHenry,IL,US,60050,60050,...,SESE,HO,FMF,NaN,HO,NaN,Traditional,N,NaN,"SYRINGE, DISPOSABLE, GLASS & PLASTIC"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23311,K955593,SHIPPERT MEDICAL TECHNOLOGIES CORP.,SARAH LAKE,"7002 SOUTH REVERE PARKWAY,",SUITE 60,ENGLEWOOD,CO,US,80112,80112,...,SESE,EN,EMX,Summary,EN,NaN,Traditional,N,NaN,EXPANDACELL ANATOMICAL POSTERIOR NASAL PACK W/...
23312,K955609,COLOPLAST CORP.,M. SYDNEY LILLY,1955 WEST OAK CIR.,NaN,MARIETTA,GA,US,30062-2249,30062-2249,...,SESE,GU,FAQ,Statement,GU,NaN,Traditional,N,NaN,CONVEEN URINE BAG - STERILE
23313,K955613,MISTEN MEDICAL CORP.,GEORGE W MURRAY,7220 STOUT RD.,NaN,GERMANTOWN,TN,US,38138,38138,...,SESE,EN,ETB,Statement,EN,NaN,Traditional,N,NaN,MISTEN MEDICAL PARTIAL OSSICULAR REPLACEMENT P...
23314,K955615,MISTEN MEDICAL CORP.,GEORGE W MURRAY,7220 STOUT RD.,NaN,GERMANTOWN,TN,US,38138,38138,...,SESE,EN,ETA,Statement,EN,NaN,Traditional,N,NaN,MISTEN MEDICAL TOTAL OSSICULAR REPLACEMENT PRO...
